In [3]:
import os.path as osp

import torch
import torch.nn.functional as F

import torch_geometric as tg
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GraphConv, TopKPooling
from torch_geometric.nn import global_max_pool as gmp
from torch_geometric.nn import global_mean_pool as gap
import sys

sys.path.append("../")
import utils

train_dataset = utils.GraphDataset(
    "../data/", "MixedShapesSmallTrain_TRAIN", True, n_quantiles=100
)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = utils.GraphDataset(
    "../data/", "MixedShapesSmallTrain_TEST", True, n_quantiles=100
)
test_loader = DataLoader(test_dataset, batch_size=64)


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        nn = tg.nn.MLP([1, 64])
        self.conv1 = tg.nn.GINConv(nn)
        self.pool1 = TopKPooling(64, ratio=0.8)
        nn = tg.nn.MLP([64, 64])
        self.conv2 = tg.nn.GINConv(nn)
        self.pool2 = TopKPooling(64, ratio=0.8)
        nn = tg.nn.MLP([64, 64])
        self.conv3 = tg.nn.GINConv(nn)
        self.pool3 = TopKPooling(64, ratio=0.8)

        self.lin1 = torch.nn.Linear(128, 64)
        self.lin2 = torch.nn.Linear(64, 16)
        self.lin3 = torch.nn.Linear(16, 5)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.3, training=self.training)
        x = F.relu(self.lin2(x))
        x = F.log_softmax(self.lin3(x), dim=-1)

        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=3, mode="min", cooldown=1, factor=0.5, verbose=True
)


def train(epoch):
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        pred = model(data).max(dim=1)[1]

        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)


for epoch in range(200):
    loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    scheduler.step(loss)
    print(
        f"Epoch: {epoch:03d}, Loss: {loss:.5f}, Train Acc: {train_acc:.5f}, "
        f"Test Acc: {test_acc:.5f}"
    )



Epoch: 000, Loss: 1.66425, Train Acc: 0.20000, Test Acc: 0.27134
Epoch: 001, Loss: 1.63038, Train Acc: 0.21000, Test Acc: 0.28866
Epoch: 002, Loss: 1.61610, Train Acc: 0.27000, Test Acc: 0.39340
Epoch: 003, Loss: 1.60226, Train Acc: 0.27000, Test Acc: 0.42062
Epoch: 004, Loss: 1.60519, Train Acc: 0.23000, Test Acc: 0.32536
Epoch: 005, Loss: 1.57502, Train Acc: 0.26000, Test Acc: 0.39216
Epoch: 006, Loss: 1.58937, Train Acc: 0.36000, Test Acc: 0.46639
Epoch: 007, Loss: 1.56956, Train Acc: 0.38000, Test Acc: 0.46598
Epoch: 008, Loss: 1.56812, Train Acc: 0.38000, Test Acc: 0.47381
Epoch: 009, Loss: 1.58385, Train Acc: 0.26000, Test Acc: 0.34515
Epoch: 010, Loss: 1.53109, Train Acc: 0.28000, Test Acc: 0.33443
Epoch: 011, Loss: 1.51700, Train Acc: 0.39000, Test Acc: 0.45567
Epoch: 012, Loss: 1.52111, Train Acc: 0.34000, Test Acc: 0.37320
Epoch: 013, Loss: 1.52980, Train Acc: 0.34000, Test Acc: 0.44165
Epoch: 014, Loss: 1.50593, Train Acc: 0.37000, Test Acc: 0.45897
Epoch: 015, Loss: 1.490